In [2]:
import math
import io
import torch
from torchvision import transforms
import numpy as np
import torch.nn.functional as F
from PIL import Image

def _compute_psnr(a, b, max_val: float = 1.0) -> float:
    mse = torch.mean((a - b) ** 2).item()
    psnr = 20 * np.log10(max_val) - 10 * np.log10(mse)
    return psnr


import matplotlib.pyplot as plt
device = 'cuda' if torch.cuda.is_available() else 'cpu'
img = Image.open('./stmalo_fracape.png').convert('RGB')
x = transforms.ToTensor()(img).unsqueeze(0).to(device)

theta = torch.rand(1).unsqueeze(0).to(device)
theta = theta * 0.5 + 0.5
# theta = torch.tensor(0.9607).to(device)
# theta = tehta.reshape(1,1)
# theta = torch.ones((1,1)).to(device)
theta0 = torch.zeros_like(theta, dtype=torch.float32).to(device)
print(theta)
stn_matrix = torch.cat((1/theta, theta0, theta0, theta0, 1/theta, theta0), axis=1)
stn_revmatrix = torch.cat((theta, theta0, theta0, theta0, theta, theta0), axis=1)

B, C, H, W = x.shape
rh = torch.full(theta.size(), H, dtype=torch.int32).to(device)
rw = torch.full(theta.size(), W, dtype=torch.int32).to(device)
ch = torch.minimum(torch.floor(H * theta + 4).to(torch.int32).to(device), rh)
cw = torch.minimum(torch.floor(W * theta + 4).to(torch.int32).to(device), rw)
ph1 = ((rh - ch) / 2).to(torch.int32)
ph2 = (rh - ch - ph1).to(torch.int32)
pw1 = ((rw - cw) / 2).to(torch.int32)
pw2 = (rw - cw - pw1).to(torch.int32)

grid = F.affine_grid(stn_matrix.view(1, 2, 3), x.size())
grid1 = F.affine_grid(stn_matrix.view(1,2,3), [B, C, ch, cw])

y = F.grid_sample(x, grid, mode='bicubic')
if ph2 > 0 and pw2 > 0:
    z = y[:, :, ph1:-ph2, pw1:-pw2]
    z = F.pad(z, (pw1, pw2, ph1, ph2))
else:
    z = y
grid_rev = F.affine_grid(stn_revmatrix.view(1, 2, 3), z.size())

w = F.grid_sample(z, grid_rev, mode='bicubic')
w1 = F.grid_sample(y, grid_rev, mode='bicubic')

y1 = F.grid_sample(x, grid1, mode='bicubic')
grid1_rev = F.affine_grid(stn_revmatrix.view(1, 2, 3), z.size())
w2 = F.grid_sample(y1, grid1_rev, mode='bicubic')


print(y1.size())
print(z.size())
print('psnr %f'%(_compute_psnr(x, w)))
print('psnr %f'%(_compute_psnr(x, w1)))
print('psnr %f'%(_compute_psnr(x, w2)))


y = transforms.ToPILImage()(y.squeeze().cpu())
z = transforms.ToPILImage()(z.squeeze().cpu())
w = transforms.ToPILImage()(w.squeeze().cpu())
# w2 = transforms.ToPILImage()(w2.squeeze().cpu())
b = F.interpolate(x, (int(H * theta), int(W * theta)), mode='bicubic')
c = F.interpolate(b, (H, W), mode='bicubic')
print('psnr %f'%(_compute_psnr(x, c)))
c = transforms.ToPILImage()(c.squeeze().cpu())

%matplotlib inline
plt.figure(figsize=(12, 9))
plt.axis('off')
plt.imshow(img)
plt.show()
plt.figure(figsize=(12, 9))
plt.axis('off')
plt.imshow(transforms.ToPILImage()(y1.squeeze().cpu()))
plt.show()
plt.figure(figsize=(12, 9))
plt.axis('off')
plt.imshow(transforms.ToPILImage()(w2.squeeze().cpu()))
plt.show()

# w2 = F.interpolate(y[:, :, ph1:-ph2, pw1:-pw2], (H, W), mode='bicubic')

FileNotFoundError: [Errno 2] No such file or directory: '/home/jianghao/Code/Graduation/CompressAI/examples/assets/stmalo_fracape.png'